In [ ]:
import os
import sys
import glob
import h5py
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
#scarica il dataset modelnet40 usato nell'articolo DGCNN

!wget --no-check-certificate https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip
!unzip modelnet40_ply_hdf5_2048.zip
!mv modelnet40_ply_hdf5_2048 data/
!rm -f modelnet40_ply_hdf5_2048.zip

--2021-05-08 11:46:12--  https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 435212151 (415M) [application/zip]
Saving to: ‘modelnet40_ply_hdf5_2048.zip’

modelnet40_ply_hdf5 100%[===================>] 415.05M  2.05MB/s    in 2m 56s  

2021-05-08 11:49:09 (2.35 MB/s) - ‘modelnet40_ply_hdf5_2048.zip’ saved [435212151/435212151]

Archive:  modelnet40_ply_hdf5_2048.zip
   creating: modelnet40_ply_hdf5_2048/
  inflating: modelnet40_ply_hdf5_2048/ply_data_train_2_id2file.json  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train2.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train4.h5  
  inflating: modelnet40_ply_hdf5_2048/ply_data_train1.h5  
  inflating: modelnet40_ply_hdf5_2048/train_files.txt  
  inflat

In [ ]:
#legge dati e label
def load_data_cls(partition):
    DATA_DIR = 'data'
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(DATA_DIR,'*%s*.h5'%partition)):
        f = h5py.File(h5_name, 'r+')
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label

In [ ]:
# funzioni per dataugmentation (random traslation, rotations, jitter delle pointlcoud)
def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud

def rotate_pointcloud(pointcloud):
    theta = np.pi*2 * np.random.uniform()
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    pointcloud[:,[0,2]] = pointcloud[:,[0,2]].dot(rotation_matrix) # random rotation (x,z)
    return pointcloud

def jitter_pointcloud(pointcloud, sigma=0.01, clip=0.02):
    N, C = pointcloud.shape
    pointcloud += np.clip(sigma * np.random.randn(N, C), -1*clip, clip)
    return pointcloud

In [ ]:
# clase per leggere il dataset e formattarlo in modo appopriato per l'input della DGCNN 
# il dataset contiene tre tipi di oggetti:
# points: la point cloud che contiene le coordinate dei punti: shape (N, P, C_p) N=numero esempi, P=numero punti per ogni esempio, C_p=numero feature associate a ciascun punto
# features: le feature associate ad ogni punto (possono essere le stesse coordinate o queste + ulteriori features): shape (N, P, C_f)
# mask: una mask che ha valore 1 o 0 per mascherare punti non fisici (quando P di un dato evento è inferiore alla dimensione con cui si e' fissato P): shape (N,P,1) 

x(8000,4096,7) # x,y,z,vx,vy,vz,tipo
y(8000,200,3)

inport train_test_split from sklearn 

x_train, x_test, y_train, y_test = train_test_split(x,y, shuffle=True, random_seed=1234)

x_train
x_test
y_train
y_test
... 

class Dataset(object):
    def __init__(self, partition='train', num_points=4096): 
        if partition == 'train':
          self.data, self.label = x_train,y_train
        else
          self.data, self.label = x_test,y_test
        self.num_points = num_points
        self._values = {}
        self._label = None
        self._load()
        
    def __len__(self):
        return len(self._label)

    def _load(self):
        pointcloud = self.data[:, :4096, :3]
        mask = np.ones(shape=(pointcloud.shape[0],pointcloud.shape[1],1))
        features = self.data[:, :4096, :] 
        self._label = self.label[:,[0,100,199],1] --> (8000,3)
        self._values['points'] = pointcloud
        self._values['features'] = features
        self._values['mask'] = mask
    
    def __getitem__(self, key):
        if key=='label':
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]  

In [ ]:
#legge i dati di training e test
train = Dataset(partition='train', num_points=4096)
test = Dataset(partition='test', num_points=4096)

In [ ]:
# mostra il contenuto dei dati
print(train['points'].shape)
print(test['points'].shape)
print(train['features'].shape)
print(test['features'].shape)
print(train['mask'].shape)
print(test['mask'].shape)

(9840, 1024, 3)
(2468, 1024, 3)
(9840, 1024, 3)
(2468, 1024, 3)
(9840, 1024, 1)
(2468, 1024, 1)


In [ ]:
#DGCNN

# A shape is (N, P_A, C), B shape is (N, P_B, C)
# D shape is (N, P_A, P_B)
def batch_distance_matrix_general(A, B):
    with tf.name_scope('dmat'):
        r_A = tf.reduce_sum(A * A, axis=2, keepdims=True)
        r_B = tf.reduce_sum(B * B, axis=2, keepdims=True)
        m = tf.matmul(A, tf.transpose(B, perm=(0, 2, 1)))
        D = r_A - 2 * m + tf.transpose(r_B, perm=(0, 2, 1))
        return D
    
def knn(num_points, k, topk_indices, features):
    # topk_indices: (N, P, K)
    # features: (N, P, C)
    with tf.name_scope('knn'):
        queries_shape = tf.shape(features)
        batch_size = queries_shape[0]
        batch_indices = tf.tile(tf.reshape(tf.range(batch_size), (-1, 1, 1, 1)), (1, num_points, k, 1))
        indices = tf.concat([batch_indices, tf.expand_dims(topk_indices, axis=3)], axis=3)  # (N, P, K, 2)
        return tf.gather_nd(features, indices)
    
def edge_conv(points, features, num_points, K, channels, with_bn=True, activation='relu', pooling='max', name='edgeconv'):
    """EdgeConv
    Args:
        K: int, number of neighbors
        in_channels: # of input channels
        channels: tuple of output channels
        pooling: pooling method ('max' or 'average')
    Inputs:
        points: (N, P, C_p)
        features: (N, P, C_0)
    Returns:
        transformed points: (N, P, C_out), C_out = channels[-1]
    """

    with tf.name_scope('edgeconv'):

        # distance
        D = batch_distance_matrix_general(points, points)  # (N, P, P)
        _, indices = tf.nn.top_k(-D, k=K + 1)  # (N, P, K+1)
        indices = indices[:, :, 1:]  # (N, P, K)

        fts = features
        knn_fts = knn(num_points, K, indices, fts)  # (N, P, K, C)
        knn_fts_center = tf.tile(tf.expand_dims(fts, axis=2), (1, 1, K, 1))  # (N, P, K, C)
        knn_fts = tf.concat([knn_fts_center, tf.subtract(knn_fts, knn_fts_center)], axis=-1)  # (N, P, K, 2*C)

        x = knn_fts
        for idx, channel in enumerate(channels):
            x = keras.layers.Conv2D(channel, kernel_size=(1, 1), strides=1, data_format='channels_last',
                                    use_bias=False if with_bn else True, kernel_initializer='glorot_normal', name='%s_conv%d' % (name, idx))(x)
            if with_bn:
                x = keras.layers.BatchNormalization(name='%s_bn%d' % (name, idx))(x)
            if activation:
                x = keras.layers.Activation(activation, name='%s_act%d' % (name, idx))(x)

        if pooling == 'max':
            fts = tf.reduce_max(x, axis=2)  # (N, P, C')
        else:
            fts = tf.reduce_mean(x, axis=2)  # (N, P, C')

        # shortcut
        sc = keras.layers.Conv2D(channels[-1], kernel_size=(1, 1), strides=1, data_format='channels_last',
                                 use_bias=False if with_bn else True, kernel_initializer='glorot_normal', name='%s_sc_conv' % name)(tf.expand_dims(features, axis=2))
        if with_bn:
            sc = keras.layers.BatchNormalization(name='%s_sc_bn' % name)(sc)
        sc = tf.squeeze(sc, axis=2)

        if activation:
            return keras.layers.Activation(activation, name='%s_sc_act' % name)(sc + fts)  # (N, P, C')
        else:
            return sc + fts


def _DGCNN_base(points, features=None, mask=None, setting=None, name='DGCNN_SG'):
    # points : (N, P, C_coord)
    # features:  (N, P, C_features), optional
    # mask: (N, P, 1), optional

    with tf.name_scope(name):
        if features is None:
            features = points
        
        if mask is not None:
            mask = tf.cast(tf.not_equal(mask, 0), dtype='float32')  # 1 if valid
            coord_shift = tf.multiply(999., tf.cast(tf.equal(mask, 0), dtype='float32'))  # make non-valid positions to 99   
            
        fts = tf.squeeze(keras.layers.BatchNormalization(name='%s_fts_bn' % name)(tf.expand_dims(features, axis=2)), axis=2)
        for layer_idx, layer_param in enumerate(setting.conv_params):
            K, channels = layer_param
            pts = tf.add(coord_shift, points) if layer_idx == 0 else tf.add(coord_shift, fts)
            fts = edge_conv(pts, fts, setting.num_points, K, channels, with_bn=True, activation='relu',
                            pooling=setting.conv_pooling, name='%s_%s%d' % (name, 'EdgeConv', layer_idx))

        if mask is not None:
            fts = tf.multiply(fts, mask)

        pool = tf.reduce_mean(fts, axis=1)  # (N, C)

        if setting.fc_params is not None:
            x = pool
            for layer_idx, layer_param in enumerate(setting.fc_params):
                units, drop_rate = layer_param
                x = keras.layers.Dense(units, activation='relu')(x)
                if drop_rate is not None and drop_rate > 0:
                    x = keras.layers.Dropout(drop_rate)(x)


            #out = keras.layers.Dense(setting.num_class, activation='softmax')(x)
            out = keras.layers.Dense(setting.num_class)(x)
            return out  # (N, num_classes)
        else:
            return pool


class _DotDict:
    pass

def get_DGCNN(num_classes, input_shapes):
    """
    Parameters
    ----------
    num_classes : int
        Number of output classes.
    input_shapes : dict
        The shapes of each input (`points`, `features`, `mask`).
    """
    setting = _DotDict()
    setting.num_class = num_classes
    # conv_params: list of tuple in the format (K, (C1, C2, C3))
    setting.conv_params = [
        (20, (64, 64, 64)),
        (20, (64, 64, 64)),
        (20, (128, 128, 128)),
        (20, (256, 256, 256)),
        ]
    # conv_pooling: 'average' or 'max'
    setting.conv_pooling = 'max'
    # fc_params: list of tuples in the format (C, drop_rate)
    setting.fc_params = [
        (512, 0.5),
        (256, 0.5),
        ]
    setting.num_points = input_shapes['points'][0]

    points = keras.Input(name='points', shape=input_shapes['points'])
    features = keras.Input(name='features', shape=input_shapes['features']) if 'features' in input_shapes else None
    mask = keras.Input(name='mask', shape=input_shapes['mask']) if 'mask' in input_shapes else None
    outputs = _DGCNN_base(points, features, mask, setting, name='DGCNN_SG')

    return keras.Model(inputs=[points, features, mask], outputs=outputs, name='DGCNN_SG')


In [ ]:
num_classes = 3
input_shapes = {k:train[k].shape[1:] for k in train.X}
print(input_shapes)
model = get_DGCNN(num_classes, input_shapes)

{'points': (1024, 3), 'features': (1024, 3), 'mask': (1024, 1)}


In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    logging.info('Learning rate: %f'%lr)
    return lr

In [ ]:
model.compile(loss='mse',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['mae'])
model.summary()

[2021-05-08 13:38:48,009] INFO: Learning rate: 0.001000


Model: "DGCNN_SG"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 1024, 1)]    0                                            
__________________________________________________________________________________________________
tf.math.not_equal_1 (TFOpLambda (None, 1024, 1)      0           mask[0][0]                       
__________________________________________________________________________________________________
tf.cast_2 (TFOpLambda)          (None, 1024, 1)      0           tf.math.not_equal_1[0][0]        
__________________________________________________________________________________________________
tf.math.equal_1 (TFOpLambda)    (None, 1024, 1)      0           tf.cast_2[0][0]                  
___________________________________________________________________________________________

In [ ]:
# Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = 'DGCNN_modelbest.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_mae',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [ ]:
# Training parameters
batch_size = 32
epochs = 30

In [ ]:
train.shuffle()
model.fit(train.X, train.y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test.X, test.y),
          shuffle=True,
          callbacks=callbacks)

[2021-05-08 13:39:57,292] INFO: Learning rate: 0.001000


Epoch 1/30
      0/Unknown - 144s 0s/sample - loss: 2.9062 - accuracy: 0.2718
Epoch 00001: val_accuracy improved from -inf to 0.19854, saving model to model_checkpoints/DGCNN_modelbest.h5
308/308 [==============================] - 157s 511ms/sample - loss: 2.6481 - accuracy: 0.3239 - val_loss: 3.3160 - val_accuracy: 0.1985


[2021-05-08 13:42:34,750] INFO: Learning rate: 0.001000


Epoch 2/30
  0/308 [..............................] - ETA: 0s - loss: 1.6588 - accuracy: 0.5188
Epoch 00002: val_accuracy improved from 0.19854 to 0.57455, saving model to model_checkpoints/DGCNN_modelbest.h5
308/308 [==============================] - 155s 503ms/sample - loss: 1.6083 - accuracy: 0.5334 - val_loss: 1.4749 - val_accuracy: 0.5746


[2021-05-08 13:45:09,707] INFO: Learning rate: 0.001000


...

Epoch 28/30
  0/308 [..............................] - ETA: 0s - loss: 0.3629 - accuracy: 0.8823
Epoch 00028: val_accuracy improved from 0.88695 to 0.88776, saving model to model_checkpoints/DGCNN_modelbest.h5
308/308 [==============================] - 155s 503ms/sample - loss: 0.3634 - accuracy: 0.8827 - val_loss: 0.3763 - val_accuracy: 0.8878


[2021-05-08 14:52:17,155] INFO: Learning rate: 0.000100


Epoch 29/30
  0/308 [..............................] - ETA: 0s - loss: 0.3554 - accuracy: 0.8821
Epoch 00029: val_accuracy did not improve from 0.88776
308/308 [==============================] - 155s 503ms/sample - loss: 0.3603 - accuracy: 0.8820 - val_loss: 0.3806 - val_accuracy: 0.8825


[2021-05-08 14:54:51,982] INFO: Learning rate: 0.000100


Epoch 30/30
  0/308 [..............................] - ETA: 0s - loss: 0.3793 - accuracy: 0.8761
Epoch 00030: val_accuracy improved from 0.88776 to 0.89182, saving model to model_checkpoints/DGCNN_modelbest.h5
308/308 [==============================] - 155s 503ms/sample - loss: 0.3701 - accuracy: 0.8797 - val_loss: 0.3691 - val_accuracy: 0.8918


In [ ]:
model.load_weights("model_checkpoints/DGCNN_modelbest.h5")

In [ ]:
test_loss, test_mae = model.evaluate(test.X, test.y, verbose=2)

NameError: ignored

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pred = model.predict(test.X)
matrix = confusion_matrix(test.y.argmax(axis=1), pred.argmax(axis=1), normalize='true')

In [ ]:
#confusion matrix
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()

    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.2f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [ ]:
le_label=[ str(i) for i in range(test.y.shape[1])]
print_cm(matrix,le_label)

              0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15    16    17    18    19    20    21    22    23    24    25    26    27    28    29    30    31    32    33    34    35    36    37    38    39 
        0  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
        1  0.00  0.90  0.08  0.00  0.00  0.00  0.02  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
        2  0.00  0.00  0.99  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.01  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.